# ES Order Data Quality Analysis

This notebook pulls ALL active ES Orders and categorizes them by data issues for remediation.

## Filters Applied
- **Status Filter:** Activated, Suspended (Late Payment), Disconnect in Progress
- **PA Market Exclusion:** Pittsburgh, Harrisburg, Philadelphia, Scranton, Uniti-PA

## Output: Excel file with tabs:
1. **Summary** - Overall counts and issues
2. **Ready_To_Migrate** - Orders with all required fields populated
3. **Missing_BAN** - Orders missing Billing_Invoice__c (CRITICAL - cannot migrate)
4. **Missing_A_Location** - Orders missing Address_A__c
5. **Missing_Node** - Orders missing Node__c (can be post-migration)
6. **Missing_Service_Start** - Orders missing Service_Start_Date__c
7. **EXCLUDED_PA_Market** - Orders excluded due to PA market
8. **All_Active_Orders** - Complete list of all active orders (after PA filter)

---
**Created:** December 11, 2024

In [1]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime
import os

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Imports successful")

Python: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Pandas: 2.2.3
✅ Imports successful


In [3]:
# === CONFIGURATION ===

# ES (Source) Credentials
ES_USERNAME = "sfdcapi@everstream.net"
ES_PASSWORD = "pV4CAxns8DQtJsBq!"
ES_TOKEN = "r1uoYiusK19RbrflARydi86TA"
ES_DOMAIN = "login"  # or 'login' for production

# Active Status Filter
ACTIVE_STATUSES = ["Activated", "Suspended (Late Payment)", "Disconnect in Progress"]

# PA Market Filter (EXCLUDE these values from Dimension_4_Market__c)
PA_MARKETS_TO_EXCLUDE = [
    "Pittsburgh",
    "Harrisburg",
    "Philadelphia",
    "Scranton",
    "Uniti-PA",
]

# Output settings
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_FILE = f"es_orders_data_quality_{TIMESTAMP}.xlsx"

print("📋 Configuration loaded")
print(f"   Active Statuses: {ACTIVE_STATUSES}")
print(f"   PA Markets to EXCLUDE: {PA_MARKETS_TO_EXCLUDE}")
print(f"   Output: {OUTPUT_FILE}")

📋 Configuration loaded
   Active Statuses: ['Activated', 'Suspended (Late Payment)', 'Disconnect in Progress']
   PA Markets to EXCLUDE: ['Pittsburgh', 'Harrisburg', 'Philadelphia', 'Scranton', 'Uniti-PA']
   Output: es_orders_data_quality_20251230_092815.xlsx


In [4]:
# === CONNECT TO ES SALESFORCE ===

print("🔌 Connecting to ES Salesforce...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

🔌 Connecting to ES Salesforce...
✅ Connected to ES: everstream.my.salesforce.com


In [5]:
# === QUERY ALL ACTIVE ORDERS ===

print("📊 Querying ALL active Orders (this may take a moment)...")

# Build status filter
status_filter = "','".join(ACTIVE_STATUSES)

# Query all fields we need for analysis and migration
orders_query = f"""
SELECT 
    Id, 
    Name,
    Service_ID__c,
    Status,
    AccountId,
    Account.Name,
    Billing_Invoice__c,
    Address_A__c,
    Address_Z__c,
    Node__c,
    OpportunityId,
    Service_Start_Date__c,
    Service_End_Date__c,
    Service_Provided__c,
    SOF_MRC__c,
    OSS_Service_ID__c,
    OSS_Order__c,
    Vendor_Circuit_ID__c,
    Primary_Product_Family__c,
    Primary_Product_Name__c,
    Dimension_4_Market__c,
    CreatedDate,
    LastModifiedDate
FROM Order
WHERE Status IN ('{status_filter}')
ORDER BY Service_ID__c
"""

try:
    result = es_sf.query_all(orders_query)
    orders_df = pd.DataFrame(result["records"])

    # Flatten Account.Name
    if "Account" in orders_df.columns:
        orders_df["Account_Name"] = orders_df["Account"].apply(
            lambda x: x.get("Name") if isinstance(x, dict) else None
        )
        orders_df = orders_df.drop(columns=["Account", "attributes"], errors="ignore")
    else:
        orders_df = orders_df.drop(columns=["attributes"], errors="ignore")

    print(f"\n✅ Retrieved {len(orders_df):,} active Orders")
    print(f"\n=== Status Breakdown ===")
    print(orders_df["Status"].value_counts().to_string())

except Exception as e:
    print(f"❌ Query error: {e}")
    raise

📊 Querying ALL active Orders (this may take a moment)...

✅ Retrieved 17,947 active Orders

=== Status Breakdown ===
Status
Activated                   17674
Disconnect in Progress        272
Suspended (Late Payment)        1


In [6]:
# === FILTER OUT PA MARKET ===

print("🔍 Filtering out PA Market orders...")

# Identify PA market orders
pa_market_mask = orders_df["Dimension_4_Market__c"].isin(PA_MARKETS_TO_EXCLUDE)
pa_orders_df = orders_df[pa_market_mask].copy()
orders_df = orders_df[~pa_market_mask].copy()

print(f"\n=== PA Market Filter Results ===")
print(f"   Orders EXCLUDED (PA Market): {len(pa_orders_df):,}")
print(f"   Orders REMAINING (to migrate): {len(orders_df):,}")

if len(pa_orders_df) > 0:
    print(f"\n=== PA Market Breakdown ===")
    print(pa_orders_df["Dimension_4_Market__c"].value_counts().to_string())

# Also show market distribution for remaining orders
print(f"\n=== Remaining Orders by Market ===")
print(
    orders_df["Dimension_4_Market__c"].value_counts(dropna=False).head(15).to_string()
)

🔍 Filtering out PA Market orders...

=== PA Market Filter Results ===
   Orders EXCLUDED (PA Market): 1,054
   Orders REMAINING (to migrate): 16,893

=== PA Market Breakdown ===
Dimension_4_Market__c
Uniti-PA        577
Philadelphia    283
Pittsburgh       90
Scranton         75
Harrisburg       29

=== Remaining Orders by Market ===
Dimension_4_Market__c
Cleveland                   5963
Michigan                    3179
Detroit                     2006
Indiana                     1532
Wisconsin                   1185
Columbus                    1162
Illinois                     971
Wisconsin Expansion          523
Missouri                     299
Kentucky                      64
None                           7
Multi - Expansion Market       2


In [7]:
# === CATEGORIZE ORDERS BY DATA ISSUES ===

print("🔍 Categorizing Orders by data issues...")

# Create issue flags
orders_df["Missing_BAN"] = orders_df["Billing_Invoice__c"].isna()
orders_df["Missing_A_Location"] = orders_df["Address_A__c"].isna()
orders_df["Missing_Z_Location"] = orders_df["Address_Z__c"].isna()
orders_df["Missing_Node"] = orders_df["Node__c"].isna()
orders_df["Missing_Service_Start"] = orders_df["Service_Start_Date__c"].isna()
orders_df["Missing_Opportunity"] = orders_df["OpportunityId"].isna()

# Calculate issue count per order
issue_cols = [
    "Missing_BAN",
    "Missing_A_Location",
    "Missing_Z_Location",
    "Missing_Node",
    "Missing_Service_Start",
]
orders_df["Issue_Count"] = orders_df[issue_cols].sum(axis=1)

# Determine if ready to migrate (has BAN and A_Location at minimum)
orders_df["Ready_To_Migrate"] = (
    ~orders_df["Missing_BAN"] & ~orders_df["Missing_A_Location"]
)

# Summary
print("\n=== Data Quality Summary ===")
print(f"   Total Active Orders:        {len(orders_df):,}")
print(f"   ✅ Ready to Migrate:        {orders_df['Ready_To_Migrate'].sum():,}")
print(f"   ❌ NOT Ready (has issues):  {(~orders_df['Ready_To_Migrate']).sum():,}")

print("\n=== Issue Breakdown ===")
print(f"   ⚠️  Missing BAN (CRITICAL):   {orders_df['Missing_BAN'].sum():,}")
print(f"   ⚠️  Missing A Location:       {orders_df['Missing_A_Location'].sum():,}")
print(f"   ⚠️  Missing Z Location:       {orders_df['Missing_Z_Location'].sum():,}")
print(f"   ℹ️  Missing Node:             {orders_df['Missing_Node'].sum():,}")
print(f"   ℹ️  Missing Service Start:    {orders_df['Missing_Service_Start'].sum():,}")

🔍 Categorizing Orders by data issues...

=== Data Quality Summary ===
   Total Active Orders:        16,893
   ✅ Ready to Migrate:        13,433
   ❌ NOT Ready (has issues):  3,460

=== Issue Breakdown ===
   ⚠️  Missing BAN (CRITICAL):   3,447
   ⚠️  Missing A Location:       209
   ⚠️  Missing Z Location:       0
   ℹ️  Missing Node:             14,803
   ℹ️  Missing Service Start:    8,589


In [9]:
# === CREATE CATEGORY DATAFRAMES ===

print("📁 Creating category dataframes...")

# Columns to include in output (excluding flags)
output_cols = [
    "Id",
    "Service_ID__c",
    "Status",
    "Account_Name",
    "AccountId",
    "Billing_Invoice__c",
    "Address_A__c",
    "Address_Z__c",
    "Node__c",
    "OpportunityId",
    "Service_Start_Date__c",
    "Service_End_Date__c",
    "Service_Provided__c",
    "SOF_MRC__c",
    "OSS_Service_ID__c",
    "OSS_Order__c",
    "Primary_Product_Family__c",
    "Primary_Product_Name__c",
    "Dimension_4_Market__c",
    "Issue_Count",
    "Ready_To_Migrate",
]

# Filter columns that exist
output_cols = [c for c in output_cols if c in orders_df.columns]

# Ready to migrate
ready_df = orders_df[orders_df["Ready_To_Migrate"]][output_cols].copy()
print(f"   ✅ Ready_To_Migrate: {len(ready_df):,} orders")

# Missing BAN (CRITICAL)
missing_ban_df = orders_df[orders_df["Missing_BAN"]][output_cols].copy()
print(f"   ⚠️  Missing_BAN: {len(missing_ban_df):,} orders")

# Missing A Location
missing_aloc_df = orders_df[orders_df["Missing_A_Location"]][output_cols].copy()
print(f"   ⚠️  Missing_A_Location: {len(missing_aloc_df):,} orders")

# Missing Node (for reference - can be post-migration)
missing_node_df = orders_df[orders_df["Missing_Node"]][output_cols].copy()
print(f"   ℹ️  Missing_Node: {len(missing_node_df):,} orders")

# Missing Service Start Date
missing_start_df = orders_df[orders_df["Missing_Service_Start"]][output_cols].copy()
print(f"   ℹ️  Missing_Service_Start: {len(missing_start_df):,} orders")

📁 Creating category dataframes...
   ✅ Ready_To_Migrate: 13,433 orders
   ⚠️  Missing_BAN: 3,447 orders
   ⚠️  Missing_A_Location: 209 orders
   ℹ️  Missing_Node: 14,803 orders
   ℹ️  Missing_Service_Start: 8,589 orders


In [10]:
# === ANALYZE MISSING BAN ORDERS ===

print("\n🔍 Analyzing Orders missing BAN (Billing_Invoice__c)...")

if len(missing_ban_df) > 0:
    # Check if they have Account names that might give us clues
    print("\n=== Account Distribution (Missing BAN) ===")
    account_dist = missing_ban_df["Account_Name"].value_counts().head(20)
    print(account_dist.to_string())

    # Check Product Family distribution
    if "Primary_Product_Family__c" in missing_ban_df.columns:
        print("\n=== Product Family Distribution (Missing BAN) ===")
        prod_dist = missing_ban_df["Primary_Product_Family__c"].value_counts().head(10)
        print(prod_dist.to_string())

    # Check if there's a pattern (internal accounts, etc.)
    internal_keywords = ["INTERNAL", "TEST", "DEMO", "SANDBOX"]
    internal_mask = (
        missing_ban_df["Account_Name"]
        .str.upper()
        .str.contains("|".join(internal_keywords), na=False)
    )
    internal_count = internal_mask.sum()
    print(
        f"\n📌 Possibly Internal/Test accounts: {internal_count:,} ({internal_count/len(missing_ban_df)*100:.1f}%)"
    )
else:
    print("✅ No orders missing BAN!")


🔍 Analyzing Orders missing BAN (Billing_Invoice__c)...

=== Account Distribution (Missing BAN) ===
Account_Name
INTERNAL EVERSTREAM (Network Expansion Build-Out)        1423
INTERNAL EVERSTREAM (RELO & EMERGENCY REPAIR)             494
INTERNAL EVERSTREAM NETWORK RESOURCES (DO NOT DELETE)     483
INTERNAL EVS (Vendor Orders)                              186
T-Mobile Small Cell                                       111
INTERNAL EVERSTREAM (MTU BUILDINGS)                       108
Internal Everstream Network Resources Michigan             92
T-Mobile                                                   69
The Lincoln Electric Company                               35
Sherwin Williams                                           33
City of Lakewood                                           28
US Cellular - Network Engineering                          27
ACD                                                        21
DISH Wireless                                              20
Michigan Dept of Co

In [11]:
# === CREATE SUMMARY DATAFRAME ===

print("📊 Creating summary...")

total_before_pa_filter = len(orders_df) + len(pa_orders_df)

summary_data = [
    {
        "Category": "Total Active Orders (before PA filter)",
        "Count": total_before_pa_filter,
        "Percentage": "100%",
        "Action": "N/A",
    },
    {
        "Category": "🚫 EXCLUDED - PA Market",
        "Count": len(pa_orders_df),
        "Percentage": f"{len(pa_orders_df)/total_before_pa_filter*100:.1f}%",
        "Action": "Excluded from migration (Pittsburgh, Harrisburg, Philadelphia, Scranton, Uniti-PA)",
    },
    {"Category": "", "Count": "", "Percentage": "", "Action": ""},
    {
        "Category": "Total Orders to Migrate (after PA filter)",
        "Count": len(orders_df),
        "Percentage": f"{len(orders_df)/total_before_pa_filter*100:.1f}%",
        "Action": "N/A",
    },
    {
        "Category": "✅ Ready to Migrate",
        "Count": len(ready_df),
        "Percentage": f"{len(ready_df)/len(orders_df)*100:.1f}%",
        "Action": "Proceed with migration",
    },
    {
        "Category": "❌ Missing BAN (CRITICAL)",
        "Count": len(missing_ban_df),
        "Percentage": f"{len(missing_ban_df)/len(orders_df)*100:.1f}%",
        "Action": "MUST FIX - Cannot migrate without Billing_Invoice__c",
    },
    {
        "Category": "⚠️ Missing A Location",
        "Count": len(missing_aloc_df),
        "Percentage": f"{len(missing_aloc_df)/len(orders_df)*100:.1f}%",
        "Action": "Should fix - Required for A_Location__c mapping",
    },
    {
        "Category": "ℹ️ Missing Node",
        "Count": len(missing_node_df),
        "Percentage": f"{len(missing_node_df)/len(orders_df)*100:.1f}%",
        "Action": "Can fix post-migration - A_Node__c is not critical",
    },
    {
        "Category": "ℹ️ Missing Service Start",
        "Count": len(missing_start_df),
        "Percentage": f"{len(missing_start_df)/len(orders_df)*100:.1f}%",
        "Action": "Review - May need for Active_Date__c",
    },
]

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

📊 Creating summary...
                                 Category Count Percentage                                                                             Action
   Total Active Orders (before PA filter) 17947       100%                                                                                N/A
                   🚫 EXCLUDED - PA Market  1054       5.9% Excluded from migration (Pittsburgh, Harrisburg, Philadelphia, Scranton, Uniti-PA)
                                                                                                                                             
Total Orders to Migrate (after PA filter) 16893      94.1%                                                                                N/A
                       ✅ Ready to Migrate 13433      79.5%                                                             Proceed with migration
                 ❌ Missing BAN (CRITICAL)  3447      20.4%                               MUST FIX - Cannot migrate without Bil

In [12]:
# === EXPORT TO EXCEL ===

print(f"\n📁 Exporting to {OUTPUT_FILE}...")

with pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl") as writer:

    # Summary
    summary_df.to_excel(writer, sheet_name="Summary", index=False)

    # Ready to Migrate
    ready_df.to_excel(writer, sheet_name="Ready_To_Migrate", index=False)

    # Missing BAN (CRITICAL)
    missing_ban_df.to_excel(writer, sheet_name="Missing_BAN_CRITICAL", index=False)

    # Missing A Location
    missing_aloc_df.to_excel(writer, sheet_name="Missing_A_Location", index=False)

    # Missing Node (reference)
    missing_node_df.to_excel(writer, sheet_name="Missing_Node", index=False)

    # Missing Service Start
    missing_start_df.to_excel(writer, sheet_name="Missing_Service_Start", index=False)

    # PA Market Excluded Orders
    if len(pa_orders_df) > 0:
        pa_output_cols = [c for c in output_cols if c in pa_orders_df.columns]
        pa_orders_df[pa_output_cols].to_excel(
            writer, sheet_name="EXCLUDED_PA_Market", index=False
        )

    # All Active Orders (complete list - after PA filter)
    orders_df[output_cols].to_excel(writer, sheet_name="All_Active_Orders", index=False)

print(f"\n✅ Export complete: {OUTPUT_FILE}")
print(f"\n=== Sheets Created ===")
print(f"   1. Summary - Overview of data quality")
print(f"   2. Ready_To_Migrate - {len(ready_df):,} orders ready")
print(f"   3. Missing_BAN_CRITICAL - {len(missing_ban_df):,} orders (MUST FIX)")
print(f"   4. Missing_A_Location - {len(missing_aloc_df):,} orders")
print(f"   5. Missing_Node - {len(missing_node_df):,} orders (post-migration)")
print(f"   6. Missing_Service_Start - {len(missing_start_df):,} orders")
print(f"   7. EXCLUDED_PA_Market - {len(pa_orders_df):,} orders (not migrating)")
print(f"   8. All_Active_Orders - {len(orders_df):,} orders (to migrate)")


📁 Exporting to es_orders_data_quality_20251230_092815.xlsx...

✅ Export complete: es_orders_data_quality_20251230_092815.xlsx

=== Sheets Created ===
   1. Summary - Overview of data quality
   2. Ready_To_Migrate - 13,433 orders ready
   3. Missing_BAN_CRITICAL - 3,447 orders (MUST FIX)
   4. Missing_A_Location - 209 orders
   5. Missing_Node - 14,803 orders (post-migration)
   6. Missing_Service_Start - 8,589 orders
   7. EXCLUDED_PA_Market - 1,054 orders (not migrating)
   8. All_Active_Orders - 16,893 orders (to migrate)


---
## Next Steps

### Priority 1: Fix Missing BAN (3,451 orders)
These orders CANNOT be migrated without `Billing_Invoice__c`.

**Options:**
1. **Find/Create BANs** - Identify correct Billing_Invoice__c for each order
2. **Exclude from migration** - If these are internal/test records
3. **Create default BAN** - If business approves a catch-all BAN

### Priority 2: Fix Missing A Location (201 orders)
These need `Address_A__c` populated for `A_Location__c` mapping.

### Priority 3: Review Missing Node (15,811 orders)
Can be fixed post-migration since `A_Node__c` is not critical for initial load.

### Priority 4: Review Missing Service Start (8,875 orders)
May need for `Active_Date__c` - review if these are truly activated services.

In [ ]:
# === PLACEHOLDER FOR ADDITIONAL ANALYSIS ===

# Add custom queries here as needed
pass